
## Overview


# <img src="https://logos-world.net/wp-content/uploads/2020/05/NASA-Logo-1959-present.png" width="100px" align="middle" /> Introducing NASA earthaccess 🌍



#### TL;DR: [**earthaccess**](https://github.com/nsidc/earthaccess) is a Python package to search, preview and access NASA datasets (on-prem or in the cloud) with a few lines of code.


## Why? 

**Programmatic, Easy, Reproducible.**

There are many ways to access NASA datasets, we can use the [Earthdata search portal](https://search.earthaccess.nasa.gov/). We can use DAAC specific portals or tools.
We could even use [data.gov](https://data.gov)!  Web portals are great but they are not designed for programmatic access and reproducible workflows. This is extremely important in the age of the cloud and reproducible open science. 

The good news is that NASA also exposes APIs that allows us to search, transform and access data in a programmatic way. Many of these libraries contain amazing features and some similarities. In this context, **earthaccess** aims to be a simple library that can deal with the important parts of the metadata so we can access or download data without having to worry if a given dataset is on-prem or in the cloud.


## How?

> Note: There are a lot of acronyms that we need to get familiar with before any of this makes sense, here is a brief glossary for NASA Earthdata terms: [NASA glossary](glossary.md)


### Authentication: Before we can use `earthaccess` we need an account with **[NASA EDL](https://urs.earthaccess.nasa.gov/)**


Earthdata Login provides free and immediate access to thousands of EOSDIS data products covering all Earth science disciplines and topic areas for researchers, applied science users, application developers, and the general public.

Once we have our NASA EDL login credentials we can start accessing NASA data in a programmatic way.


In [ ]:
import earthaccess
earthaccess.__version__

#### Auth()

`earthaccess`'s **Auth** class provides 3 different strategies to authenticate ourselves with NASA EDL.

* **netrc**: Do we have a `.netrc` file with our EDL credentials? if so, we can use it with `earthaccess`.
If we don't have it and want to create one we can, earthaccess allows users to type their credentials and persist them into a `.netrc` file.

```python
auth = earthaccess.login(strategy="netrc")
```
* **environment**: If we have our EDL credentials as environment variables 
  * EARTHDATA_USERNAME
  * EARTHDATA_PASSWORD
```python
auth = earthaccess.login(strategy="environment")
```
* **interactive**: We will be asked for our EDL credentials with optinal persistance to `.netrc`

```python
auth = earthaccess.login(strategy="interactive", persist=True)
```



In [ ]:
# are we authenticated?

auth = earthaccess.login(strategy="netrc")


## Querying for datasets

The `DataCollections` class can query CMR for any collection (dataset) using all of CMR's Query parameters and has built-in functions to extract useful information from the response.


In [ ]:
# The first step is to create a DataCollections query 

Query = earthaccess.collection_query()

# Use chain methods to customize our query
Query.keyword('elevation').bounding_box(-134.7,58.9,-133.9,59.2).temporal("2020-01-01","2020-02-01")

print(f'Collections found: {Query.hits()}')

# filtering what UMM fields to print, to see the full record we omit the fields filters
# meta is always included as 
collections = Query.fields(['ShortName','Version']).get(5)
# Inspect some results printing just the ShortName and Abstract
collections[0:3]

The results from a DataCollections and DataGranules query are enhanced python dictionaries, this means
that we can access all the keys and values like we usually do with Python dictionaries.

In [ ]:
collections[0]["umm"]["ShortName"]

The DataCollections class returns python dictionaries with some handy methods.

```python 
collection.concept_id() # returns the concept-id, used to search for data granules
collection.abstract() # returns the abstract
collection.landing_page() # returns the landing page if present in the UMM fields
collection.get_data() # returns the portal where data can be accessed.
```

The same results can be obtained using the `dict` syntax:

```python
collection["meta"]["concept-id"] # concept-id
collection["umm"]["RelatedUrls"] # URLs, with GET DATA, LANDING PAGE etc
```


In [ ]:
# We can now search for collections using a pythonic API client for CMR.
Query = earthaccess.collection_query().daac("PODAAC")

print(f'Collections found: {Query.hits()}')
collections = Query.fields(['ShortName']).get(10)
# Printing the first collection
collections[0]

In [ ]:
# What if we want cloud collections
Query = earthaccess.collection_query().daac("PODAAC").cloud_hosted(True)

print(f'Collections found: {Query.hits()}')
collections = Query.fields(['ShortName']).get(10)
# Printing 3 collections
collections[0]

In [ ]:
# Printing the concept-id for the first 10 collections
[collection.concept_id() for collection in collections]

## Querying for data files (granules)

The DataGranules class provides similar functionality as the collection class. To query for granules in a more reliable way concept-id would be the main key.
You can search data granules using a short name but that could (more likely will) return different versions of the same data granules. 

In this example we're querying for 10 data grnaules from ICESat-2  [ATL06](https://nsidc.org/data/ATL06/versions/) version `005` dataset. 


> **Note**: Generally speaking we won't need authenticated queries unless they are restricted datasets for early adopters.

In [ ]:
from pprint import pprint

# We build our query

Query = earthaccess.granule_query().short_name('ATL06').version("005").bounding_box(-134.7,58.9,-133.9,59.2)
# We get 5 metadata records
granules = Query.get(5)
granules

## Pretty printing data granules

Since we are in a notebook we can take advantage of it to see a more user friendly version of the granules with the built-in function `display`
This will render browse image for the granule if available and eventually will have a similar representation as the one from the Earthdata search portal

In [ ]:
# printing 2 granules using display
[display(granule) for granule in granules]

### Spatiotemporal queries

Our granules and collection classes accept the same spatial and temporal arguments as CMR so we can search for granules that match spatiotemporal criteria.



In [ ]:
Query = earthaccess.granule_query().short_name("ATL06").temporal("2020-03-01", "2020-03-30").bounding_box(-134.7,58.9,-133.9,59.2).version("005")
# Always inspects the hits before retrieven the granule metadata, just because it's very verbose.
print(f"Granules found: {Query.hits()}")

In [ ]:
# Now we can print some info about these granules using the built-in methods
granules = Query.get(5)
data_links = [{'links': g.data_links(access="on_prem"), 'size (MB):': g.size()} for g in granules]
data_links

## **Accessing the data**

With `earthaccess` a researcher can get the files regardless if they are on-prem or cloud based with the same API call, although an important consideration is that if we want to access data in the cloud (direct access) we must run the code in the cloud. This is because some S3 buckets are configured to only allow direct access (s3:// links) if the requester is in the same zone, `us-west-2`.

## On-prem access: DAAC hosted data  📡


For this example we are going to use a PODAAC dataset `SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5` which we previously queried (see querying for datasets) and got the concept id: `C1972955240-PODAAC`

In [ ]:
import earthaccess

earthaccess.login()

Query = earthaccess.granule_query().short_name("SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5").bounding_box(-134.7,54.9,-100.9,69.2)
print(f"Granule hits: {Query.hits()}")
# getting more than 6,000 metadata records for demo purposes is going to slow us down a bit so let's get only a few
granules = Query.get(10)
# Does this granule belong to a cloud-based collection?
granules[0].cloud_hosted

### Finally! let's get the data

The Store class accepts the results from a `DataGranules()` query or it can also accept a list of URLs for the data files. In the second case we'll have to specify the DAAC since it cannot infer which credentials to use solely on the URL.

In [ ]:
%%time
files = earthaccess.download(granules[0:4], "./data/C1972955240-PODAAC/")

## **Accessing the data in the cloud ☁️** 
<img src="https://www.freecodecamp.org/news/content/images/size/w2000/2020/08/Screenshot-2020-08-10-at-6.26.31-PM.png" width="300px" align="middle" >



With `earthaccess` a researcher can get the files regardless if they are on-prem or cloud based with the same API call, although an important consideration is that if we want to access data in the cloud we must run the code in the cloud. This is because some S3 buckets are configured to only allow direct access (s3:// links) if the requester is in the same zone, `us-west-2`.

Same API, just a different place, in this case the `concept-id` for the same dataset is `C2208422957-POCLOUD`
> Note: The `concept-id` changed even though is the same dataset.

In [ ]:

Query = earthaccess.granule_query().short_name("SMAP_JPL_L3_SSS_CAP_8DAY-RUNNINGMEAN_V5").bounding_box(-134.7,54.9,-100.9,69.2)
print(f"Granule hits: {Query.hits()}")
results = Query.get(10)
# is this a cloud hosted data granule?
results[0].cloud_hosted

In [ ]:
# Let's pretty print this
results[0]

In [ ]:
%%time
# we are running this code outside the us-west-2 region.
try:
    files = earthaccess.download(results[0:4], local_path="./data/demo-POCLOUD")
except Exception as e:
    print(f"Error: {e}, we are probably not using this code in the Amazon cloud. Trying external links...")
    # There is hope, even if we are not in the Amazon cloud we can still get the data
    files = store.get(results[0:4], access="external", local_path="./data/demo-POCLOUD")

##  ☁️ **Cloud Access Part II: streaming data**

Being in the cloud allows us to stream data as if we were using it locally. Pairing gridded datasets on S3 and xarray isa very useful patter when we deal with a lot of data. 

> **Recommended read: [Skip the download! Stream NASA data directly into Python objects](https://medium.com/pangeo/intake-stac-nasa-4cd78d6246b7)**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import xarray as xr
# data_links
https_links = []
s3_links = []

fs = earthaccess.get_s3fs_session(provider='POCLOUD')

for granule in results:
    https_links.extend(granule.data_links(access="on_prem"))
    s3_links.extend(granule.data_links(access="direct"))
s3_links

### With earthaccess we can open URLS if we pass the provider or we can open the results directly and will derive the necessary credentials to use.

In [ ]:
%%time

ds_L3 = xr.open_mfdataset(
    earthaccess.open(results[0:3]),
    combine='nested',
    concat_dim='time',
    coords='minimal',
    )
ds_L3

## Now to the important science! 🚀 

### Recap

```python

import earthaccess

# first we authenticate with NASA EDL
auth = earthaccess.login()

# Then we build a Query with spatiotemporal parameters
GranuleQuery = earthaccess.granule_query().concept_id("C1575731655-LPDAAC_ECS").bounding_box(-134.7,58.9,-133.9,59.2)

# We get the metadata records from CMR
granules = GranuleQuery.get()

# Now it{s time to download (or open) our data granules list with get()
files = earthaccess.download(granules, local_path='./data')

# Now to the important science!
```

### Related links

**Github repository**: https://github.com/nsidc/earthaccess

**CMR** API documentation: https://cmr.earthaccess.nasa.gov/search/site/docs/search/api.html

**EDL** API documentation: https://urs.earthaccess.nasa.gov/

NASA OpenScapes: https://nasa-openscapes.github.io/earthaccess-cloud-cookbook/

NSIDC: https://nsidc.org


Contact: luis.lopez@nsidc.org